In [ ]:
import cv2
import numpy as np
import pandas as pd

# Загрузка изображения
image = cv2.imread('ecg.png')

# Преобразование в черно-белое изображение
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Коррекция перспективы
def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

pts = np.array([[30, 60], [450, 50], [460, 300], [40, 320]], dtype="float32")
warped = four_point_transform(gray, pts)

# Удаление теней
dilated_img = cv2.dilate(warped, np.ones((7,7), np.uint8))
bg_img = cv2.medianBlur(dilated_img, 21)
diff_img = 255 - cv2.absdiff(warped, bg_img)
norm_img = cv2.normalize(diff_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)

# Удаление углов (смятие бумаги)
def remove_corners(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.ones(img.shape, dtype="uint8") * 255

    for c in contours:
        if cv2.contourArea(c) < 1000:
            cv2.drawContours(mask, [c], -1, 0, -1)

    img = cv2.bitwise_or(img, img, mask=mask)
    return img

norm_img = remove_corners(norm_img)

# Применение адаптивного порога
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Увеличение резкости границ
sharp_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
sharp_img = cv2.filter2D(adaptive_thresh, -1, sharp_kernel)

# Преобразование в бинарное изображение
_, binary_img = cv2.threshold(sharp_img, 127, 255, cv2.THRESH_BINARY)

# Выделение и удаление сетки
def remove_grid(img):
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    horizontal_lines = cv2.morphologyEx(img, cv2.MORPH_OPEN, horizontal_kernel)

    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    vertical_lines = cv2.morphologyEx(img, cv2.MORPH_OPEN, vertical_kernel)

    mask = cv2.add(horizontal_lines, vertical_lines)
    img_no_grid = cv2.bitwise_not(cv2.bitwise_or(img, mask))
    return img_no_grid

binary_img = remove_grid(binary_img)

# Детектирование сигнала ЭКГ и его извлечение
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
ecg_contour = max(contours, key=cv2.contourArea)

x, y, w, h = cv2.boundingRect(ecg_contour)
ecg_signal = binary_img[y:y+h, x:x+w]

# Объединение сигнала
def extract_signal(img):
    signal = []
    height, width = img.shape
    for col in range(width):
        column = img[:, col]
        indices = np.where(column == 0)[0]
        if len(indices) > 0:
            signal.append(height - indices.mean())
        else:
            signal.append(height / 2)
    return signal

signal = extract_signal(ecg_signal)

# Вывод результата в формат .csv
df = pd.DataFrame(signal, columns=['ECG Signal'])
df.to_csv('ecg_signal.csv', index=False)

print("Экспорт завершен успешно.")